In [1]:
# CNN5

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
from math import e
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.model_selection import KFold

In [3]:
def model(X_train, Y_train, X_val, Y_val):
    model = Sequential()
    if model_choice == 'one':
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same', input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3],1), data_format='channels_first'))
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same'))
        model.add(AveragePooling2D(pool_size=2,padding='same'))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(32, kernel_size=3,padding='same', activation='relu'))
        model.add(Conv2D(32, kernel_size=3,padding='same', activation='relu'))
        
        model.add(AveragePooling2D(pool_size=2,padding='same'))
        model.add(Dropout({{uniform(0, 1)}}))
    
    elif model_choice == 'two':
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3],1), data_format='channels_first'))
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same'))
        model.add(AveragePooling2D(pool_size=2,padding='same'))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(64, kernel_size=3,padding='same', activation='relu'))
        model.add(Conv2D(64, kernel_size=3,padding='same', activation='relu'))
        
        model.add(AveragePooling2D(pool_size=2,padding='same'))
        model.add(Dropout({{uniform(0, 1)}}))
        
    model.add(Flatten())
    model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
    
    model.add(Dropout({{uniform(0, 1)}}))
    choiceval = {{choice(['one', 'two'])}}
    if choiceval == 'two':
        model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
        
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(units=1, activation = 'linear'))
    
    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer='adam', loss='mae',  metrics='accuracy')
    model.fit(X_train, Y_train,epochs=20,verbose=2,
              validation_data=(X_val, Y_val))
    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}